In [1]:
#IMPORT
import numpy as np
import scipy as sp
import sys
import os
import pandas as pd
from pathlib import Path
import pickle

current_dir = Path().resolve()

src_path = current_dir.parent / "src"

# Add to sys.path
sys.path.append(str(src_path))

from tvb.simulator.lab import *
import functions_simulator as fun
import os
import matplotlib.pylab as plt
import matplotlib.cm as cm
from scipy import stats
import time
import Utils as fx

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

In [2]:
# Define the retry function
def retry_function(func, attempts=5, delay=1):
    for _ in range(attempts):
        try:
            return func()
        except Exception as e:
            print(f"Attempt failed: {e}")
            time.sleep(delay)
    # If all attempts fail, return None
    return None

# Run simulations with homogeneous tuning of local parameters 

In [7]:
a_values=[0.967, 0.973, 0.979] # ~[supercritical, critical, subcritical]

In [3]:
# CONNECTIVITY
conn_file = parent_directory+'/data/Allen_148/Allen_148.zip' #'/Allen_connectivity.zip'
Allen_SC = fun.set_up_connectivity148(conn_file, np.infty)
SC=Allen_SC.weights
nroi=len(SC)
Cortical_labels=np.array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
                          21, 22, 23, 24, 25, 26, 27, 28, 29, 74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  
                          84,  85,  86, 87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103])

Working_labels=Cortical_labels

len(Working_labels)

In [5]:
Allen_SC.tract_lengths=Allen_SC.tract_lengths[np.ix_(Working_labels,Working_labels)]
Allen_SC.weights=Allen_SC.weights[np.ix_(Working_labels,Working_labels)]
Allen_SC.number_of_regions=len(Working_labels)

# Structural Info
df = pd.read_csv('../data/Allen_148/Allen_rois_nets.csv', index_col=0)

# Subcritical Regime

In [9]:
G_par=np.linspace(0.,0.21,16)

mean_speed=np.infty #3 #np.mean(Allen_SC.tract_lengths)/3   # conduction speed

Fixed_parameters_hybrid={'J':1.25, 
                         'a':a_values[2], 
                         'sigma':0.6725, 
                         'omega':1., 
                         'v':mean_speed, 
                         'dt':.05, 
                         'sim_length':180000,
                         'N':0.0001,
                         'noise_seed':0,
                         'Gs':G_par}

J = Fixed_parameters_hybrid['J']
a = Fixed_parameters_hybrid['a']
sigma = Fixed_parameters_hybrid['sigma']
omega = Fixed_parameters_hybrid['omega']
v = Fixed_parameters_hybrid['v']
sim_length = Fixed_parameters_hybrid['sim_length']
dt = Fixed_parameters_hybrid['dt'] 
NN=Fixed_parameters_hybrid['N'] 
noise_seed=Fixed_parameters_hybrid['noise_seed'] 

In [10]:
# In this folder we save the whole-brain simulations
out_folder='my_out_folder/Gscan_connectome_sub/'
with open(out_folder+'Fixed_parameters_hybrid.pkl', 'wb') as file:
    pickle.dump(Fixed_parameters_hybrid, file)

In [18]:
import time

ini = 0
for iG, G in enumerate(G_par[ini:]):
    iG = iG + ini
    print(iG, end='\r')
    # Start timer
    start_time = time.time()

    
    
    # Define the function to be retried
    def call_function():
        # Your function call here
        t_b, B_r, B_psi, t, r, psi, input = fun.simulate_network_conduct_seed_wholeBOLD_R(Allen_SC, J, a, sigma, omega, G, NN, noise_seed, v, sim_length, dt, verbose=True)
        return t_b, B_r, B_psi, t, r, psi, input
    
    # Retry the function call
    result = retry_function(call_function, attempts=25, delay=1)
    
    if result is not None:
        t_b, B_r, B_psi, t, r, psi, input = result

        BOLD_R=B_r[:,:]
        BOLD_PSI=B_psi[:,:]
        R=r[::10,:]
        PSI=psi[::10,:]
        INPUT=input[::10,:]
        my_dict = {'BOLD_R':BOLD_R,'BOLD_PSI':BOLD_PSI, 'R':R,'PSI':PSI,'INPUT':INPUT}

        with open(out_folder + 'data_G%d.pkl'%iG, 'wb') as file:
            pickle.dump(my_dict, file)
    
    else:
        print(f"Function call failed for G value {G}. Skipping...")
    
    # End timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"1-loop execution time: {elapsed_time:.4f} seconds")

1-loop execution time: 1639.0204 seconds
1-loop execution time: 1660.6523 seconds
1-loop execution time: 1766.2576 seconds
1-loop execution time: 1778.4859 seconds
1-loop execution time: 1640.9314 seconds
1-loop execution time: 1586.1518 seconds
1-loop execution time: 1630.8870 seconds
1-loop execution time: 1573.0581 seconds
1-loop execution time: 1609.6915 seconds
1-loop execution time: 1600.0531 seconds
1-loop execution time: 1618.1334 seconds
1-loop execution time: 1597.9761 seconds
1-loop execution time: 1597.6493 seconds
1-loop execution time: 1584.6357 seconds
1-loop execution time: 1600.9418 seconds
1-loop execution time: 1557.9843 seconds


In [17]:
R.shape

(36000, 60)

In [18]:
BOLD_R.shape

(18, 60)

# Critical Regime

In [11]:
G_par=np.linspace(0.,0.21,16)

mean_speed=np.infty #3 #np.mean(Allen_SC.tract_lengths)/3   # conduction speed

Fixed_parameters_hybrid={'J':1.25, 
                         'a':a_values[1], 
                         'sigma':0.6725, 
                         'omega':1., 
                         'v':mean_speed, 
                         'dt':.05, 
                         'sim_length':180000,
                         'N':0.0001,
                         'noise_seed':0,
                         'Gs':G_par}

J = Fixed_parameters_hybrid['J']
a = Fixed_parameters_hybrid['a']
sigma = Fixed_parameters_hybrid['sigma']
omega = Fixed_parameters_hybrid['omega']
v = Fixed_parameters_hybrid['v']
sim_length = Fixed_parameters_hybrid['sim_length']
dt = Fixed_parameters_hybrid['dt'] 
NN=Fixed_parameters_hybrid['N'] 
noise_seed=Fixed_parameters_hybrid['noise_seed'] 

In [12]:
# In this folder we save the whole-brain simulations
out_folder='my_out_folder/Gscan_connectome_crit/'
with open(out_folder+'Fixed_parameters_hybrid.pkl', 'wb') as file:
    pickle.dump(Fixed_parameters_hybrid, file)

In [21]:
import time

ini = 0
for iG, G in enumerate(G_par[ini:]):
    iG = iG + ini
    print(iG, end='\r')
    # Start timer
    start_time = time.time()

    
    
    # Define the function to be retried
    def call_function():
        # Your function call here
        t_b, B_r, B_psi, t, r, psi, input = fun.simulate_network_conduct_seed_wholeBOLD_R(Allen_SC, J, a, sigma, omega, G, NN, noise_seed, v, sim_length, dt, verbose=True)
        return t_b, B_r, B_psi, t, r, psi, input
    
    # Retry the function call
    result = retry_function(call_function, attempts=25, delay=1)
    
    if result is not None:
        t_b, B_r, B_psi, t, r, psi, input = result

        BOLD_R=B_r[:,:]
        BOLD_PSI=B_psi[:,:]
        R=r[::10,:]
        PSI=psi[::10,:]
        INPUT=input[::10,:]
        my_dict = {'BOLD_R':BOLD_R,'BOLD_PSI':BOLD_PSI, 'R':R,'PSI':PSI,'INPUT':INPUT}

        with open(out_folder + 'data_G%d.pkl'%iG, 'wb') as file:
            pickle.dump(my_dict, file)
    
    else:
        print(f"Function call failed for G value {G}. Skipping...")
    
    # End timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"1-loop execution time: {elapsed_time:.4f} seconds")

1-loop execution time: 1596.5640 seconds
1-loop execution time: 1619.4083 seconds
1-loop execution time: 1576.9633 seconds
1-loop execution time: 1627.2347 seconds
1-loop execution time: 1614.2655 seconds
1-loop execution time: 1595.9987 seconds
1-loop execution time: 1616.7601 seconds
1-loop execution time: 1575.5948 seconds
1-loop execution time: 1563.0708 seconds
1-loop execution time: 1611.6232 seconds
1-loop execution time: 1580.3707 seconds
1-loop execution time: 1601.5522 seconds
1-loop execution time: 1587.5316 seconds
1-loop execution time: 1577.3266 seconds
1-loop execution time: 1676.2258 seconds
1-loop execution time: 1676.1971 seconds


# Supercritical Regime

In [11]:
G_par=np.linspace(0.,0.21,16)

mean_speed=np.infty 

Fixed_parameters_hybrid={'J':1.25, 
                         'a':a_values[0], 
                         'sigma':0.6725, 
                         'omega':1., 
                         'v':mean_speed, 
                         'dt':.05, 
                         'sim_length':180000,
                         'N':0.0001,
                         'noise_seed':0,
                         'Gs':G_par}

J = Fixed_parameters_hybrid['J']
a = Fixed_parameters_hybrid['a']
sigma = Fixed_parameters_hybrid['sigma']
omega = Fixed_parameters_hybrid['omega']
v = Fixed_parameters_hybrid['v']
sim_length = Fixed_parameters_hybrid['sim_length']
dt = Fixed_parameters_hybrid['dt'] 
NN=Fixed_parameters_hybrid['N'] 
noise_seed=Fixed_parameters_hybrid['noise_seed'] 

In [13]:
# In this folder we save the whole-brain simulations
out_folder='my_out_folder/Gscan_connectome_super/'
with open(out_folder+'Fixed_parameters_hybrid.pkl', 'wb') as file:
    pickle.dump(Fixed_parameters_hybrid, file)

In [13]:
import time

ini = 0
for iG, G in enumerate(G_par[ini:]):
    iG = iG + ini
    print(iG, end='\r')
    # Start timer
    start_time = time.time()

    
    
    # Define the function to be retried
    def call_function():
        # Your function call here
        t_b, B_r, B_psi, t, r, psi, input = fun.simulate_network_conduct_seed_wholeBOLD_R(Allen_SC, J, a, sigma, omega, G, NN, noise_seed, v, sim_length, dt, verbose=True)
        return t_b, B_r, B_psi, t, r, psi, input
    
    # Retry the function call
    result = retry_function(call_function, attempts=25, delay=1)
    
    if result is not None:
        t_b, B_r, B_psi, t, r, psi, input = result

        BOLD_R=B_r[:,:]
        BOLD_PSI=B_psi[:,:]
        R=r[::10,:]
        PSI=psi[::10,:]
        INPUT=input[::10,:]
        my_dict = {'BOLD_R':BOLD_R,'BOLD_PSI':BOLD_PSI, 'R':R,'PSI':PSI,'INPUT':INPUT}

        with open(out_folder + 'data_G%d.pkl'%iG, 'wb') as file:
            pickle.dump(my_dict, file)
    
    else:
        print(f"Function call failed for G value {G}. Skipping...")
    
    # End timer
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"1-loop execution time: {elapsed_time:.4f} seconds")

1-loop execution time: 1665.3645 seconds
1-loop execution time: 2476.3382 seconds
1-loop execution time: 2429.5182 seconds
1-loop execution time: 1940.9817 seconds
1-loop execution time: 2248.4267 seconds
1-loop execution time: 1771.0570 seconds
1-loop execution time: 1813.3377 seconds
1-loop execution time: 1708.5966 seconds
1-loop execution time: 1689.9251 seconds
1-loop execution time: 1586.2337 seconds
1-loop execution time: 1592.0849 seconds
1-loop execution time: 1588.0719 seconds
1-loop execution time: 1592.3670 seconds
1-loop execution time: 1583.3789 seconds
1-loop execution time: 1585.4488 seconds
1-loop execution time: 1606.6868 seconds
